###### Importing necessery libraries

In [1]:
import numpy as np

In [2]:
import pandas as pd

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [4]:
from sklearn.metrics.pairwise import cosine_similarity

In [5]:
import pickle

In [6]:
#load the dataset
music_data = pd.read_csv("Music.csv")

In [11]:
#combine relevent features for content based filtering
music_data['combined_features'] = music_data['Singer/Artists'] + " " + music_data['Genre'] + " " + music_data['Album/Movie']

In [42]:
#Handle missing valu
music_data['combined_features'].isna()

0       False
1       False
2       False
3       False
4       False
        ...  
2415    False
2416    False
2417    False
2418    False
2419    False
Name: combined_features, Length: 2407, dtype: bool

In [43]:
music_data['combined_features'].isna().sum()

0

In [44]:
music_data['combined_features'].dropna(inplace=True)

In [45]:
music_data['combined_features']

0       Kumar Sanu, Mika Singh, Neha Kakkar BollywoodD...
1       Neha Kakkar, Tony Kakkar BollywoodDanceRomanti...
2                  Ranveer Singh BollywoodDance Gully Boy
3       Jyotica Tangri, Shaan, Subhro Ganguly Bollywoo...
4       Asees Kaur, Rahat Fateh Ali Khan, Tanishk Bagc...
                              ...                        
2415                        Mukesh BollywoodDance Sasural
2416    Lata Mangeshkar BollywoodDance Jab Pyar Kisi S...
2417    Lata Mangeshkar, Suresh Wadkar BollywoodDance ...
2418         Jagjit Kaur BollywoodDance Shola Aur Shabnam
2419         Asha Bhosle BollywoodDance Shola Aur Shabnam
Name: combined_features, Length: 2407, dtype: object

In [46]:
music_data['combined_features'].isna().sum()

0

In [47]:
# Remove rows with null values in the 'combined_features' column
music_data = music_data.dropna(subset=['combined_features'])

In [48]:
music_data['combined_features'].isna().sum()

0

In [49]:
# Apply TF-IDF to the combined features
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(music_data['combined_features'])

In [50]:
# Compute cosine similarity
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [51]:
# Create a series to map song names to indices
indices = pd.Series(music_data.index, index=music_data['Song-Name']).drop_duplicates()

In [52]:
# Updated recommendation function to return song names
def get_recommendations(song_name, cosine_sim=cosine_sim):
    idx = indices[song_name]  # Get the index of the song in the dataset
    sim_scores = list(enumerate(cosine_sim[idx]))  # Get pairwise similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)  # Sort by similarity score
    sim_scores = sim_scores[1:6]  # Get top 5 similar songs (ignoring the first because it's the song itself)
    
    # Get the song indices for the top 5 recommendations
    song_indices = [i[0] for i in sim_scores]
    
    # Return the actual song names
    return music_data['Song-Name'].iloc[song_indices].tolist()


In [53]:
import pickle

model_to_save = (tfidf, cosine_sim, indices, music_data)

# Save the model to a file
with open('music_recommendation_model.pkl', 'wb') as model_file:
    pickle.dump(model_to_save, model_file)

print("Model saved successfully!")


Model saved successfully!


In [54]:
import pickle

# Load the saved model
with open('music_recommendation_model.pkl', 'rb') as model_file:
    tfidf, cosine_sim, indices, music_data = pickle.load(model_file)


In [55]:
import pickle

# tfidf, cosine_sim, indices, music_data

# Prepare a tuple to save
model_to_save = (tfidf, cosine_sim, indices)

# Save the model to a file
with open('music_recommendation_model.pkl', 'wb') as model_file:
    pickle.dump(model_to_save, model_file)

print("Model saved successfully!")


Model saved successfully!


In [56]:
# Test the recommendation system
print(get_recommendations('Mere Gully Mein'))

['Mere Gully Mein', 'Doori', 'Meri Aawargi', 'Wat Wat Wat', 'Sooraj Dooba Hai Yaron (Remix)']


In [57]:

# Apply TF-IDF to the combined features
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(music_data['combined_features'])

# Calculate cosine similarity matrix
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Save song indices for later use
indices = pd.Series(music_data.index, index=music_data['Song-Name']).drop_duplicates()

# Save the model (TF-IDF, cosine similarity, indices, and data)
model_to_save = (tfidf, cosine_sim, indices, music_data)
with open('music_recommendation_model.pkl', 'wb') as model_file:
    pickle.dump(model_to_save, model_file)

print("Model saved successfully!")

Model saved successfully!
